Testing SuperK SELECT control via Python. We are using NKT Photonics SuperK Compact laser with SELECT tunable filter. Laser is on COM3.

On Rayleigh, SDK documents can be found under Users/Public/Public Documents/NKT Photonics

SuperK Compact info is on page 59 of the user manual.

Python interface doesn't work while Control software is connected.

In [1]:
# testing

In [6]:
from controls import *

In [7]:
scan_ports()

Following ports has modules: COM3
Comport: COM3 Device type: 0x0074 at address: 1
Comport: COM3 Device type: 0x0067 at address: 16
Comport: COM3 Device type: 0x0066 at address: 17
Close result:  0:OPSuccess


In [8]:
get_interlock()

Waiting for interlock to reset.


In [9]:
reset_interlock()

Interlock is OK.


In [13]:
trig_mode()

Laser mode: Internal frequency generator


In [10]:
emission_on()

Setting emission ON. 0:RegResultSuccess


In [12]:
overall_power(20)

Setting overall power level to 20%.


In [14]:
emission_off()

Setting emission OFF. 0:RegResultSuccess


In [ ]:
# Monitor 1 readout, not working?
result = registerReadU16('COM3', 16, 0x10, -1)
print(result)
print(RegisterResultTypes(result))

(0, 4095)
Unknown result


In [21]:
COMport = 'COM3' # depends on the port the device is connected to. COM3 for Rayleigh desktop
COMPACT_devID = 1 # fixed for the SuperK COMPACT
SELECT_devID = 16 # fixed for the SuperK SELECT

def display_backlight(brightness=None):
    if brightness is None:
        result = registerReadU32(COMport, COMPACT_devID, 0x26, -1)
        backlight_level = result[1]
        print('Display backlight level: ', backlight_level, '%')
    else:
        result = registerWriteReadU32(COMport, COMPACT_devID, 0x26, brightness, -1)
        backlight_level = result[1]
        print(f'Display backlight level set to {backlight_level}%.')        

display_backlight()

Display backlight level:  70 %


In [ ]:
# Monitor 2 readout, not working?
result = registerReadU16('COM3', 17, 0x11, -1)
print(result)
print(RegisterResultTypes(result))

(0, 350)
Unknown result


In [70]:
# crystal 1 minimum wavelength, value in pm
result = registerReadU32('COM3', 16, 0x90, -1)
print(result)
print(RegisterResultTypes(result))

(0, 500000)
Unknown result


In [ ]:
def interlock_status():
    if value==0:
        print('Interlock off')
    if value

In [ ]:
status_mapping = {
    0: "Device is off.",
    1: "Device is on.",
    2: "Device is broken."
}

# Function to check the status based on the mapping
def check_device_status(value):
    # Use the get method to provide a default message for unknown values
    message = status_mapping.get(value, "Unknown status.")
    print(message)

# Example usage
status_value = int(input("Enter the device status (0: off, 1: on, 2: broken): "))
check_device_status(status_value)

In [ ]:
interlock_mapping = {
    0: "Interlock off (interlock circuit open).",
    1: "Waiting for interlock reset.",
    3: "Interlock is OK."
}

def interlock_status(value):
    message = interlock_mapping.get(value, "Unknown interlock status.")
    print(message)

 0 Interlock off (interlock circuit open)
0 1 Waiting for interlock reset
0 2 Interlock is OK
1 0 Front panel interlock / key switch off
2 0 Door switch open
3 0 External module interlock
4 0 Interlock power failure
255 - Interlock circuit failure

"All wavelength and amplitude settings are sent to the RF Driver, not to the SuperK SELECT"
First channel is controlled with 0x90, second with 0x91, and so on up to 0x97 for the eighth channel. 
Only use the element on index 0. Can send only one 32 bit integer to change this value. 

In [ ]:
COMport = 'COM3' # depends on the port the device is connected to
COMPACT_devID = 1 # fixed for the SuperK COMPACT
SELECT_devID = 16 # fixed for the SuperK SELECT

def crystal_temp():
    """Read the current crystal temperature in degrees Celsius."""
    result = registerReadS16(COMport, SELECT_devID, 0x38, -1)
    temperature = result[1] # temperature given in 0.1 degrees C
    print('Crystal temperature: ', temperature/10, 'degrees C.', RegisterResultTypes(result))

In [ ]:
def set_wavelength(channel, wavelength):
    """Specify the channel (1-8) and set the wavelength in nm for that channel."""
    channel = int(channel-1)
    address = f"0x9{channel}"
    result = registerWriteReadU32(COMport, SELECT_devID, address, wavelength*1000, -1)
    print('Wavelength set to: ', wavelength/1000, 'nm.', RegisterResultTypes(result))

In [1]:
# testing